### SQL 생성 프롬프트

In [1]:
def make_prompt(ddl, question, query=""):
    prompt = f"""
당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.
### DDL:
{ddl}

 ### Question:
{question}

### SQL:
{query}
"""

    return prompt

In [2]:
example1 = make_prompt(
    ddl="""CREATE TABLE messages (
        "message_id" SERIAL PRIMARY KEY,
        "conversation_id" INT NOT NULL,
        "sender_id" INT NOT NULL,
        "content" TEXT,
        "timestamp" TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
        "read" BOOLEAN DEFAULT FALSE,
        FOREIGN KEY ("conversation_id") REFERENCES conversations("conversation_id"),
        FOREIGN KEY ("sender_id") REFERENCES users("user_id")
    );""",
    question="messages 테이블에서 모든 데이터를 조회해 줘",
    query="SELECT * FROM messages;"
)
print(example1)


당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.
### DDL:
CREATE TABLE messages (
        "message_id" SERIAL PRIMARY KEY,
        "conversation_id" INT NOT NULL,
        "sender_id" INT NOT NULL,
        "content" TEXT,
        "timestamp" TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
        "read" BOOLEAN DEFAULT FALSE,
        FOREIGN KEY ("conversation_id") REFERENCES conversations("conversation_id"),
        FOREIGN KEY ("sender_id") REFERENCES users("user_id")
    );

 ### Question:
messages 테이블에서 모든 데이터를 조회해 줘

### SQL:
SELECT * FROM messages;



### GPT-4 평가 프롬프트와 코드 준비

In [3]:
import json
from pathlib import Path


def make_requests_for_gpt_evaluation(df, filename, dir='requests'):
    if not Path(dir).exists():
        Path(dir).mkdir(parents=True)
    prompts = []
    for idx, row in df.iterrows():
        prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Questions.
If gen_sql do equal job, return "yes" else return "no".
Output JSON Format: {"resolve_yn": ""}""" + f"""

DDL: {row['context']}
Question: {row['question']}
gt_sql: {row['answer']}
gen_sql: {row['gen_sql']}"""
)
    jobs = [
        {
            "model": "gpt-4-turbo-preview", 
            "response_format": {"type": "json_object"},
            "messages": [{"role": "system", "content": prompt}]
        }
        for prompt in prompts
    ]
    with open(Path(dir, filename), 'w') as f:
        for job in jobs:
            json_string = json.dumps(job)
            f.write(json_string + '\n')

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "Your OpenAI API key"
"""
python api_request_parallel_processor.py \
    --requests_filepath {요청 파일 경로} \
    --save_filepath {생성할 결과 파일 경로} \
    --request_url https://api.openai.com/v1/chat/completions \
    --max_requests_per_minute 300 \
    --max_tokens_per_minute 100000 \
    --token_encoding_name cl100k_base \
    --max_attempts 5 \
    --logging_level 20
"""

'\npython api_request_parallel_processor.py     --requests_filepath {요청 파일 경로}     --save_filepath {생성할 결과 파일 경로}     --request_url https://api.openai.com/v1/chat/completions     --max_requests_per_minute 300     --max_tokens_per_minute 100000     --token_encoding_name cl100k_base     --max_attempts 5     --logging_level 20\n'

In [4]:
import pandas as pd

def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
    prompts = []
    responses =[]
    with open(input_file, 'r') as json_file:
        for data in json_file:
            prompts.append(json.loads(data)[0]['messages'][0]['content'])
            responses.append(json.loads(data)[1]['choices'][0]['message']['content'])
    
    df = pd.DataFrame({prompt_column: prompts, response_column: responses})
    df.to_csv(output_file, index=False)
    return df

### 미세 조정 수행하기

In [6]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

def make_inference_pipeline(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return pipe

model_name = 'beomi/Yi-Ko-6B'
hf_pipe = make_inference_pipeline(model_name)

example = """당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE players (
    player_id INT PRIMARY KEY AUTO_INCREMENT,
    username VARCHAR(255) UNIQUE NOT NULL,
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    date_joined DATETIME NOT NULL,
    last_login DATETIME
);

 ### Question:
사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.

### SQL:
"""

hf_pipe(example, do_sample=False, return_full_text=False, max_length=1024, truncation=True)
# SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';
# 
# ### SQL 봇:
# SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';
# 
# ### SQL 봇의 결과:
# 사용자 이름에 'admin'이 포함되어 있는 계정의 수는 1개입니다.

2024-11-15 14:07:19.668909: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 14:07:19.668982: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 14:07:19.866230: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 14:07:20.318441: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-15 14:07:23.488693: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

[{'generated_text': "SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\n사용자 이름에 'admin'이 포함되어 있는 계정의 수는 1개입니다."}]

In [7]:
from datasets import load_dataset

# 데이터셋 불러오기
df = load_dataset("shangrilar/ko_text2sql", "origin")['test']
df = df.to_pandas()
for idx, row in df.iterrows():
    prompt = make_prompt(row['context'], row['question'])
    df.loc[idx, 'prompt'] = prompt
# sql 생성
gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False,
                                return_full_text=False, max_length=1024, truncation=True)
gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
df['gen_sql'] = gen_sqls

In [ ]:
# 평가를 위한 requests.jsonl 생성
eval_filepath = "text2sql_evaluation.jsonl"
make_requests_for_gpt_evaluation(df, eval_filepath)
# GPT-4 평가 수행
!python api_request_parallel_processor.py \
    --requests_filepath requests/{eval_filepath} \
    --save_filepath results/{eval_filepath} \
    --request_url https://api.openai.com/v1/chat/completions \
    --max_requests_per_minute 2500 \
    --max_tokens_per_minute 100000 \
    --token_encoding_name cl100k_base \
    --max_attempts 5 \
    --logging_level 20

INFO:root:Starting request #0
INFO:root:Starting request #1
INFO:root:Starting request #2
INFO:root:Starting request #3
INFO:root:Starting request #4
INFO:root:Starting request #5
INFO:root:Starting request #6
INFO:root:Starting request #7
INFO:root:Starting request #8
INFO:root:Starting request #9
INFO:root:Starting request #10
INFO:root:Starting request #11
INFO:root:Starting request #12
INFO:root:Starting request #13
INFO:root:Starting request #14
INFO:root:Starting request #15
INFO:root:Starting request #16
INFO:root:Starting request #17
INFO:root:Starting request #18
INFO:root:Starting request #19
INFO:root:Starting request #20
INFO:root:Starting request #21
INFO:root:Starting request #22
INFO:root:Starting request #23
INFO:root:Starting request #24
INFO:root:Starting request #25
INFO:root:Starting request #26
INFO:root:Starting request #27
INFO:root:Starting request #28
INFO:root:Starting request #29
INFO:root:Starting request #30
INFO:root:Starting request #31
INFO:root:Starting

In [ ]:
df_sql = load_dataset("shangrilar/ko_text2sql", "origin")['train']
df_sql = df_sql.to_pandas()
df_sql = df_sql.dropna().sample(frac=1, random_state=42)
df_sql = df_sql.query("db_id != 1")

for idx, row in df_sql.iterrows():
    df_sql.loc[idx, 'text'] = make_prompt(row['context'], row['question'], row['answer'])

!mkdir data
df_sql.to_csv('data/train.csv', index=False)

In [ ]:
base_model = "beomi/Yi-Ko-6B"
finetuned_model = "yi-ko-6b-text2sql"

!autotrain llm \
--train \
--model {base_model} \
--project-name {finetuned_model} \
--data-path data/ \
--text-column text \
--lr 2e-4 \
--batch_size 8 \
--epochs 1 \
--block-size 1024 \
--warmup-ratio 0.1 \
--lora-r 16 \
--lora-alpha 32 \
--lora-dropout 0.05 \
--weight-decay 0.01 \
--gradient-accumulation 8 \
--mixed-precision fp16 \
--use-peft \
--quantization int4 \
--trainer sft    

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, PeftModel

model_name = base_model
device_map = {"": 0}

# LoRA와 기초 모델 파라미터 합치기
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map
)
model = PeftModel.from_pretrained(base_model, finetuned_model)
model = model.merge_and_unload()

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

# 허깅페이스 허브에 모델 및 토크나이저 저장
model.push_to_hub(finetuned_model, use_temp_dir=False)
tokenizer.push_to_hub(finetuned_model, use_temp_dir=False)

In [ ]:
model_name = "shangrilar/yi-ko-6b-text2sql"
hf_pipe = make_inference_pipeline(model_name)

hf_pipe(example, do_sample=False, return_full_text=False, max_length=1024, truncation=True)


In [ ]:
# sql 생성 수행
gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False, return_full_text=False, max_length=1024, truncation=True)
gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
df['gen_sql'] = gen_sqls

# 평가를 위한 requests.jsonl 생성
eval_filepath = "text2sql_evaluation_finetuned.jsonl"
make_requests_for_gpt_evaluation(df, eval_filepath)
# GPT-4 평가 수행
!python api_request_parallel_processor.py \
    --requests_filepath requests/{eval_filepath} \
    --save_filepath results/{eval_filepath} \
    --request_url https://api.openai.com/v1/chat/completions \
    --max_requests_per_minute 2500 \
    --max_tokens_per_minute 100000 \
    --token_encoding_name cl100k_base \
    --max_attempt 5 \
    --logging_level 20

### 학습 데이터 정제와 미세 조정

In [ ]:
clean_dataset = load_dataset("shangrilar/ko_text2sql", "clean")['train']